In [1]:
# Dependencies
import requests
import os
from os.path  import basename
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import datetime
from datetime import datetime
import time
import copy
from random import randint
import unicodedata
import pprint
import json

In [2]:
leagues_df = pd.read_csv("../../data/leagues_data.csv", index_col=False)

In [3]:
soccer_data = {"Leagues": leagues_df.to_dict(orient='records')}
pprint.pprint(soccer_data)

{'Leagues': [{'country': 'Brazil',
              'league_ID': 1,
              'league_link': 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1',
              'league_name': 'Brazil Serie A',
              'tier': 1},
             {'country': 'United States',
              'league_ID': 2,
              'league_link': 'https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1',
              'league_name': 'MLS (US)',
              'tier': 1},
             {'country': 'Mexico',
              'league_ID': 3,
              'league_link': 'https://www.transfermarkt.com/liga-mx-clausura/startseite/wettbewerb/MEX1',
              'league_name': 'Liga MX (MEX)',
              'tier': 1},
             {'country': 'Argentina',
              'league_ID': 4,
              'league_link': 'https://www.transfermarkt.com/primera-division/startseite/wettbewerb/AR1N',
              'league_name': 'Superliga (ARG)',
              'tier': 1}]}


In [4]:
# to be able to scrape this website we need to use 'User Agents'
# more info about user agents in 'https://webscraping.com/blog/User-agents/'
# You can find your User-Agent at 'http://whatsmyuseragent.com/'
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

In [5]:
convert_url = "https://www.google.com/search?q=euro+to+dollar&oq=eur&aqs=chrome.1.69i57j35i39j0j46j0l2j69i61l2.2373j1j4&sourceid=chrome&ie=UTF-8"
html = requests.get(convert_url, headers=headers)
soup = BeautifulSoup(html.content, 'html.parser')
curr_value = soup.find('div', class_= "b1hJbf")
curr_value = round(float(curr_value["data-exchange-rate"]),2)

def get_value_us(x):
    value = []
    for char in x:
        value.append(char)
    if value[-1] == "m":
        float_value = "".join(value[1:-1])
        return round(float(float_value)*curr_value,2)
    else:
        float_value = "".join(value[1:-3])
        return round((float(float_value)*curr_value)/1000,2)


In [6]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text.strip())

In [7]:
def fix_heights(soccer_data1):
    for player in soccer_data1['Players']:
        
        if player["Height"] == 0:
            list_height= []
            targ_player = player
            nat = targ_player["Nat"]
            position = targ_player["position"]
            field_position = targ_player["field_position"]
#             print("Targuet Player",targ_player["name"])
            for player in soccer_data1['Players']:
                if player["position"] == position and player["Nat"] == nat:
                    if player["Height"] != 0:
#                         print(player["name"])
                        list_height.append(player["Height"])

                if len(list_height) == 0:
#                     print("lista vazia1", targ_player["name"])
                    if player["field_position"] == field_position and player["Nat"] == nat:
                        if player["Height"] != 0:
#                             print(player["name"])
                            list_height.append(player["Height"])

                if len(list_height) == 0:
#                     print("lista vazia2", targ_player["name"])
                    if player["field_position"] == field_position:
                        if player["Height"] != 0:
#                             print(player["name"])
                            list_height.append(player["Height"])

#             print("----antes",targ_player["Height"], targ_player["name"])
            targ_player["Height"] = round(sum(list_height) / len(list_height),2)
#             print(list_height)
#             print("--- depois", targ_player["name"], targ_player["Height"])


In [8]:
def scrape_league_data(soccer_data):
    team_id = 1
    
    for league in soccer_data["Leagues"]:
        teams = []
        leaguename = league['league_name']
        url = league["league_link"]
        print(f"scraping: {league['country']}_{league['tier']}_{league['league_name']}")
        time.sleep(3)
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.content, 'html.parser')
        htmltable = soup.find('table', class_= "items")
        results = htmltable.findAll("tr", class_ =["odd","even"])
#         league['clubs'] = {}
        for result in results:
            features = result.findAll("td")
            link = ("https://www.transfermarkt.com"+result.find("a", href=True)\
                          ["href"]+"/plus/1").replace("startseite", "kader")
            logo = result.find("img", src=True)["src"]
            logo = logo.split("?")[0]
            logo = logo.replace("tiny", "header")
            name = strip_accents(features[1].text)
            squad = int(features[3].text)
            foreigner = int(features[5].text)
            total_MV = float(get_value_us(features[6].text))
            avg_MV = float(get_value_us(features[7].text))
            
#             # All data
            teams.append({"team_ID" : team_id, "league_ID":league["league_ID"], 'league_name': leaguename,  "club" : name, 
                          "squad" : squad, "foreigners" : foreigner, "avg_market_m" : avg_MV,
                             "total_MV_m" : total_MV, "Logo_img" : logo, "link_page" : link})
              
#             # hierachy
#             teams.append({"team_ID" : team_id,"club" : name, "squad" : squad, "foreigners" : foreigner, "avg_market_m" : avg_MV,
#                           "total_MV_m" : total_MV, "Logo_img" : logo, "link_page" : link})
            team_id = team_id + 1
        league["Clubs"] = teams
#     soccer_data['Clubs'] = teams




In [9]:
def scrape_team_data(soccer_data):
    player_id = 1
    players = []
    for league in soccer_data["Leagues"]:
        for club in league["Clubs"]:
            team_id = club["team_ID"]
            team_link = club["link_page"]
            print(club["club"])
            time.sleep(randint(1,2))
            html = requests.get(team_link, headers=headers)
            soup = BeautifulSoup(html.content, 'html.parser')
            htmltable = soup.find('table', class_= "items")
            results = htmltable.findAll("tr", class_ =["odd","even"])

            for result in results:
                features = result.findAll("td")

                name_1 = features[2].find("img", alt=True)["alt"]

                player_page = "https://www.transfermarkt.com" + features[3].find("a", href=True)["href"]

                position_1 = features[4].text

                if position_1 in ("Centre-Back", "Left-Back", "Right-Back", "Defender"):
                    field_posit = "DEF"
                elif position_1 in ("Defensive Midfield", "Central Midfield" , "Right Midfield" , "Left Midfield" , \
                "Attacking Midfield" , "Midfield"):
                    field_posit = "MID"
                elif position_1 in ("Left Winger" , "Right Winger" , "Centre-Forward" , "Second Striker" , "Forward"):
                    field_posit = "ATT"
                else:
                    field_posit = "GLK"

                try:
                    age_1 = int((features[5].text.split("(",)[-1])[:-1])
                except:
                    age_1 = ""

                nat = features[6].img["alt"]

                try:
                    Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
                except:
                    Height_1 = 0

                foot_1 = features[8].text

                dt_joined_1 = features[9].text
                try:
                    dt_joined_1 = str(datetime.strptime(dt_joined_1, '%b %d, %Y').date())


                except:
                    dt_joined_1 = ""

                try:
                    prev_team_1 = strip_accents(features[10].img["alt"])
                except:
                    prev_team_1 = "N.A."


                contract_expires_1 = features[11].text
                try:
                    contract_expires_1 = str(datetime.strptime(contract_expires_1, '%d.%m.%Y').date())
                except:
                    contract_expires_1 = ""

                try:
                    market_value_1 = get_value_us(features[12].text[:-2])
                except:
                    market_value_1 = 0

                players.append({"players_ID": player_id, "team_ID": team_id, "league_ID":club["league_ID"], "league_name": club["league_name"], "name": name_1, "position": position_1,"field_position": field_posit, "Age":age_1,"Nat": nat, "Height": Height_1, "foot": foot_1,  'dt_joined': dt_joined_1,"prev_team": prev_team_1,\
                            "contract_expires": contract_expires_1, "market_value": market_value_1,"player_page": player_page}) 
                player_id = player_id+1
        # players dentro dos clubes (hierarchy)
#         club["Players"] = players

# players separado dos clubs #
    soccer_data["Players"] = players 
        

In [10]:
scrape_league_data(soccer_data)

scraping: Brazil_1_Brazil Serie A
scraping: United States_1_MLS (US)
scraping: Mexico_1_Liga MX (MEX)
scraping: Argentina_1_Superliga (ARG)


In [11]:
scrape_team_data(soccer_data)

Clube de Regatas do Flamengo
Sociedade Esportiva Palmeiras
Gremio Foot-Ball Porto Alegrense
Sport Club Corinthians Paulista
Sport Club Internacional
Clube Atletico Mineiro
Sao Paulo Futebol Clube
Santos FC
Fluminense Football Club
Club Athletico Paranaense
Club de Regatas Vasco da Gama
Red Bull Bragantino
Esporte Clube Bahia
Botafogo Rio de Janeiro
Goias Esporte Clube
Coritiba Foot Ball Club
Fortaleza Esporte Clube
Ceara Sporting Club
Sport Club do Recife
Atletico Clube Goianiense
Los Angeles FC
Atlanta United FC
Inter Miami CF
New York City FC
Toronto FC
Los Angeles Galaxy
Seattle Sounders FC
Columbus Crew SC
Sporting Kansas City
FC Dallas
Portland Timbers
Minnesota United FC
D.C. United
FC Cincinnati
Chicago Fire FC
Orlando City SC
Colorado Rapids
New England Revolution
Houston Dynamo
New York Red Bulls
Philadelphia Union
Montreal Impact
Vancouver Whitecaps FC
Nashville SC
San Jose Earthquakes
Real Salt Lake City
CF Monterrey
CF America
Tigres UANL
CD Cruz Azul
CF Pachuca
Deportivo G

In [12]:
# with open('testdata3.json', 'w') as fp:
#     json.dump(soccer_data, fp)

In [13]:
fix_heights(soccer_data)
with open('dict_soccer_data_hieranchy.json', 'w') as fp:
    json.dump(soccer_data, fp)

In [24]:
pprint.pprint(soccer_data)

{'Leagues': [{'Clubs': [{'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/614.png',
                         'avg_market_m': 4.81,
                         'club': 'Clube de Regatas do Flamengo',
                         'foreigners': 3,
                         'league_ID': 1,
                         'league_name': 'Brazil Serie A',
                         'link_page': 'https://www.transfermarkt.com/flamengo-rio-de-janeiro/kader/verein/614/saison_id/2019/plus/1',
                         'squad': 29,
                         'team_ID': 1,
                         'total_MV_m': 139.48},
                        {'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/1023.png',
                         'avg_market_m': 3.4,
                         'club': 'Sociedade Esportiva Palmeiras',
                         'foreigners': 4,
                         'league_ID': 1,
                         'league_name': 'Brazil Serie A',
                         'link_page': 'h

                        {'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/830.png',
                         'avg_market_m': 0.83,
                         'club': 'Club Atletico Banfield',
                         'foreigners': 6,
                         'league_ID': 4,
                         'league_name': 'Superliga (ARG)',
                         'link_page': 'https://www.transfermarkt.com/club-atletico-banfield/kader/verein/830/saison_id/2019/plus/1',
                         'squad': 38,
                         'team_ID': 79,
                         'total_MV_m': 31.39},
                        {'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/1286.png',
                         'avg_market_m': 0.85,
                         'club': "Club Atletico Newell's Old Boys",
                         'foreigners': 2,
                         'league_ID': 4,
                         'league_name': 'Superliga (ARG)',
                         'link_page': 'htt

              'prev_team': 'Santos FC',
              'team_ID': 2},
             {'Age': 25,
              'Height': 1.76,
              'Nat': 'Brazil',
              'contract_expires': '2023-12-31',
              'dt_joined': '2016-12-19',
              'field_position': 'MID',
              'foot': 'left',
              'league_ID': 1,
              'league_name': 'Brazil Serie A',
              'market_value': 2.36,
              'name': 'Raphael Veiga',
              'player_page': 'https://www.transfermarkt.com/raphael-veiga/profil/spieler/432953',
              'players_ID': 52,
              'position': 'Attacking Midfield',
              'prev_team': 'Coritiba Foot Ball Club',
              'team_ID': 2},
             {'Age': 35,
              'Height': 1.72,
              'Nat': 'Venezuela',
              'contract_expires': '2020-07-26',
              'dt_joined': '2017-01-01',
              'field_position': 'MID',
              'foot': 'right',
              'league_ID':

              'contract_expires': '2024-12-31',
              'dt_joined': '2018-01-01',
              'field_position': 'ATT',
              'foot': 'right',
              'league_ID': 1,
              'league_name': 'Brazil Serie A',
              'market_value': 8.02,
              'name': 'Pepê',
              'player_page': 'https://www.transfermarkt.com/pepe/profil/spieler/520662',
              'players_ID': 90,
              'position': 'Left Winger',
              'prev_team': 'Gremio Foot-Ball Porto Alegrense B',
              'team_ID': 3},
             {'Age': 31,
              'Height': 1.73,
              'Nat': 'Brazil',
              'contract_expires': '2022-12-31',
              'dt_joined': '2020-08-21',
              'field_position': 'ATT',
              'foot': 'left',
              'league_ID': 1,
              'league_name': 'Brazil Serie A',
              'market_value': 2.36,
              'name': 'Éverton',
              'player_page': 'https://www.transferma

              'Nat': 'Brazil',
              'contract_expires': '2022-12-31',
              'dt_joined': '2015-01-01',
              'field_position': 'MID',
              'foot': 'right',
              'league_ID': 1,
              'league_name': 'Brazil Serie A',
              'market_value': 4.72,
              'name': 'Rodrigo Dourado',
              'player_page': 'https://www.transfermarkt.com/rodrigo-dourado/profil/spieler/181780',
              'players_ID': 145,
              'position': 'Defensive Midfield',
              'prev_team': 'Sport Club Internacional B',
              'team_ID': 5},
             {'Age': 31,
              'Height': 1.82,
              'Nat': 'Brazil',
              'contract_expires': '2021-12-31',
              'dt_joined': '2019-01-10',
              'field_position': 'MID',
              'foot': 'right',
              'league_ID': 1,
              'league_name': 'Brazil Serie A',
              'market_value': 2.36,
              'name': 'Rodrigo 

...

              'position': 'Central Midfield',
              'prev_team': 'N.A.',
              'team_ID': 83},
             {'Age': 23,
              'Height': 1.79,
              'Nat': 'Argentina',
              'contract_expires': '',
              'dt_joined': '',
              'field_position': 'MID',
              'foot': '1,76 m',
              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0.0,
              'name': 'Matías Romero',
              'player_page': 'https://www.transfermarkt.com/matias-romero/profil/spieler/605721',
              'players_ID': 2787,
              'position': 'Right Midfield',
              'prev_team': 'N.A.',
              'team_ID': 83},
             {'Age': 27,
              'Height': 1.79,
              'Nat': 'Argentina',
              'contract_expires': '',
              'dt_joined': '',
              'field_position': 'MID',
              'foot': '1,68 m',
              'league_ID': 4,
        

              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0,
              'name': 'Emanuel Rosales',
              'player_page': 'https://www.transfermarkt.com/emanuel-rosales/profil/spieler/756810',
              'players_ID': 2828,
              'position': 'Left Midfield',
              'prev_team': 'N.A.',
              'team_ID': 84},
             {'Age': 31,
              'Height': 1.75,
              'Nat': 'Argentina',
              'contract_expires': '',
              'dt_joined': '',
              'field_position': 'MID',
              'foot': '1,83 m',
              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0.0,
              'name': 'Gonzalo Castellani',
              'player_page': 'https://www.transfermarkt.com/gonzalo-castellani/profil/spieler/87840',
              'players_ID': 2829,
              'position': 'Attacking Midfield',
              'prev_team': 'N.A.',
 

              'market_value': 0.0,
              'name': 'Nicolás Correa',
              'player_page': 'https://www.transfermarkt.com/nicolas-correa/profil/spieler/74361',
              'players_ID': 2878,
              'position': 'Centre-Back',
              'prev_team': 'N.A.',
              'team_ID': 86},
             {'Age': 26,
              'Height': 1.78,
              'Nat': 'Argentina',
              'contract_expires': '',
              'dt_joined': '',
              'field_position': 'DEF',
              'foot': '1,79 m',
              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0.0,
              'name': 'Jonathan Bay',
              'player_page': 'https://www.transfermarkt.com/jonathan-bay/profil/spieler/360425',
              'players_ID': 2879,
              'position': 'Left-Back',
              'prev_team': 'N.A.',
              'team_ID': 86},
             {'Age': 23,
              'Height': 1.78,
              'Na

             {'Age': 29,
              'Height': 1.79,
              'Nat': 'Uruguay',
              'contract_expires': '',
              'dt_joined': '',
              'field_position': 'DEF',
              'foot': '1,84 m',
              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0.0,
              'name': 'Mathías Abero',
              'player_page': 'https://www.transfermarkt.com/mathias-abero/profil/spieler/142370',
              'players_ID': 2914,
              'position': 'Left-Back',
              'prev_team': 'N.A.',
              'team_ID': 87},
             {'Age': 32,
              'Height': 1.78,
              'Nat': 'Argentina',
              'contract_expires': '',
              'dt_joined': '',
              'field_position': 'DEF',
              'foot': '1,76 m',
              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0.0,
              'name': 'Bruno Urribarri',
  

              'foot': '1,86 m',
              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0.0,
              'name': 'Emiliano Méndez',
              'player_page': 'https://www.transfermarkt.com/emiliano-mendez/profil/spieler/149478',
              'players_ID': 2954,
              'position': 'Defensive Midfield',
              'prev_team': 'N.A.',
              'team_ID': 88},
             {'Age': 26,
              'Height': 1.79,
              'Nat': 'Argentina',
              'contract_expires': '',
              'dt_joined': '',
              'field_position': 'MID',
              'foot': '1,82 m',
              'league_ID': 4,
              'league_name': 'Superliga (ARG)',
              'market_value': 0.0,
              'name': 'Gastón Álvarez Suarez',
              'player_page': 'https://www.transfermarkt.com/gaston-alvarez-suarez/profil/spieler/213191',
              'players_ID': 2955,
              'position': 'Central Mid

# TESTES

In [15]:
# with open('testdata5.json') as f:
#     soccer_data1 = json.load(f)



In [16]:
# for league in soccer_data1["Leagues"]:
#     clubs = []
# #     clubs = 1
#     for club in soccer_data1["Clubs"]:
# #     for league in soccer_data1["Leagues"]:
# #         clubs = 1
#         if club["league_ID"] == league["league_ID"]:
# #             print(club["club"], league["league_name"], clubs)
# #             clubs = clubs +1
# #     print(clubs, "-----")
            
#             clubs.append(club)
#     league["Clubs"] = clubs
    
# del soccer_data1["Clubs"]

In [17]:
# with open('testdata5.json', 'w') as fp:
#     json.dump(soccer_data1, fp)

## Save scraped data on SQL Database

In [18]:
# from sqlalchemy import *
# from sqlalchemy.schema import *
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.sql import text
# import contextlib 
# import json

In [19]:
# rds_connection_string = "postgres:PostgreSQL@localhost:5432/trfmk_scrape_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')
# metadata = MetaData()


In [20]:
# leagues_df = pd.read_csv("data/leagues_data.csv")
# teams_df = pd.read_csv("data/teams_trmk.csv")
# # fix 
# players_df = pd.read_csv("data/players_trmk1.csv")

FileNotFoundError: [Errno 2] File b'data/leagues_data.csv' does not exist: b'data/leagues_data.csv'

In [ ]:
# engine.execute(DropSchema('public', cascade = True))
# engine.execute('CREATE SCHEMA IF NOT EXISTS public;')

# players_df.to_sql(name = 'players',index_label= None, index= False, con=engine, if_exists='replace')
# teams_df.to_sql(name = 'teams', index_label= None, index= False, con=engine, if_exists='replace')
# leagues_df.to_sql(name = 'leagues', index_label= None, index= False, con=engine, if_exists='replace')

# with engine.connect() as con:
#     con.execute('ALTER TABLE leagues ADD CONSTRAINT pk_Leagues PRIMARY KEY ("league_ID")')
#     con.execute('ALTER TABLE teams ADD CONSTRAINT pk_Teams PRIMARY KEY ("team_ID")')
#     con.execute('ALTER TABLE players ADD CONSTRAINT pk_Players PRIMARY KEY ("players_ID")')
#     con.execute('ALTER TABLE Teams ADD CONSTRAINT fk_Teams_league_ID FOREIGN KEY("league_ID") REFERENCES Leagues ("league_ID");')
#     con.execute('ALTER TABLE Players ADD CONSTRAINT fk_Players_team_ID FOREIGN KEY("team_ID") REFERENCES Teams ("team_ID");')

In [ ]:
# # read database
# pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)

### Save query as JSON file

In [ ]:
# pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)\
# .to_json(r'dashboard/static/data/data-dict.json', orient='records')

In [ ]:
# print(json.dumps(pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine), indent = 4, sort_keys=True))

 \
 \
  \
  \
  \
  \
  \.
  

## tests

In [ ]:
# soccer_data1 = {"Leagues": [{"league_ID": 1, "Name": "Bra1", "Link": "htmls....", "Country": "Brazil", "Tier": 1},\
#                             {"league_ID": 2, "Name": "Bra2", "Link": "htmls....", "Country": "Brazil", "Tier": 2}],\
#                "Clubs": [{"team_ID" : 1, "league_ID": 1, "club" : "Flamengo", "squad" : 30, "foreigners" : 2, "avg_market_m" : 14.5,
#                      "total_MV_m" : 50, "Logo_img" : "htmls....", "link_page" : "htmls...."},
#                         {"team_ID" : 1, "club" : "GOIAS", "squad" : 25, "foreigners" : 5, "avg_market_m" : 10.5,
#                      "total_MV_m" : 40, "Logo_img" : "htmls....", "link_page" : "htmls...."}],
#                "Players":[{"players_ID": 1, "name": "Diego", "position": "GoalKeeper","field_position": "GLK", "Age":39,"Nat": "Brazil", "Height": 1.81, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 2, "name": "Marcos", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "USA", "Height": 1.82, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 13, "name": "GoalKeeper_BR", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 0, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 3, "name": "Saulo", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.83, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 4, "name": "Marcos*2", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "Brazil", "Height": 1.84, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 5, "name": "Saulo*2", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.84, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 6, "name": "Marcos**3", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "USA", "Height": 1.85, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 7, "name": "Saulo**3", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.86, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 8, "name": "Marcos***4", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "USA", "Height": 1.87, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 9, "name": "Saulo***4", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.88, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 10, "name": "Marcos****5", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "Brazil", "Height": 1.89, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 11, "name": "Saulo****5", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "USA", "Height": 1.90, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 12, "name": "Midfielder_BR", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "Brazil", "Height": 0, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
                         
#                          {"players_ID": 14, "name": "MID_USA", "position": "Middd","field_position": "MID", "Age":29,"Nat": "USA", "Height": 0, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"}
#                          ]}



In [ ]:
# teams_df = "data/BRAZIL_1_BRA_A_trmk.csv"

# scrape_team_data1(teams_df)

In [ ]:
# bra_1= pd.read_csv("data/teams_trmk.csv")
# bra_1

In [ ]:
# bra_1.dtypes

In [ ]:
# bra_1["club"] = bra_1["club"].astype(str)
# # bra_1.dtypes

In [ ]:
# files = glob('data/*.csv', recursive=True)
# scrape_team_data1(files)


In [ ]:
# df_league = pd.read_csv(teams_df)
# df_league

In [ ]:
# contratcs = ["15.06.2024", "30.12.2024", "30.06.2024"]

# for contratc in contratcs:
# #     try:
# # #         contratc_expires_1 = datetime.strptime(contratc, '%d.%m.%Y').date()
# #         dt_joined_1 = datetime.strptime(contratc, '%b %d, %Y').date()
# #     except:
# # #         contract_expires_1 = "01/01/2030"
# #         dt_joined_1 = "2019-01-01"
#     try:
#         contratc_expires_1 = datetime.strptime(contratc, '%d.%m.%Y').date()
# #         dt_joined_1 = datetime.strptime(contratc, '%b %d, %Y').date()
#     except:
#         contract_expires_1 = "2030-01-01"
# #         dt_joined_1 = "01/01/2019"

#     print(contratc_expires_1)
    

In [ ]:
# fla= pd.read_csv("data/players_trmk.csv")
# fla

In [ ]:
# fla.dtypes

In [ ]:
# flamengp = "https://www.transfermarkt.com/clube-de-regatas-do-flamengo/kader/verein/614/saison_id/2019/plus/1"
# team_name = goias.split('/')[-8]
# print(f"scraping: {team_name}")
# team_id = 1
      

    
# html = requests.get(goias, headers=headers)
# soup = BeautifulSoup(html.content, 'html.parser')
# htmltable = soup.find('table', class_= "items")

# results = htmltable.findAll("tr", class_ =["odd","even"])

# player_id = 1
# name = []
# player_page = []
# position = []
# f_posi = []
# Age = []
# Nat = []
# Height = []
# foot = []
# dt_joined = []
# prev_team = []
# contract_expires = []
# market_value = []
# team_ID = []
# players_ID = []


# for result in results:
#     features = result.findAll("td")

#     name_1 = features[2].find("img", alt=True)["alt"]

#     player_page.append("https://www.transfermarkt.com" + features[3].find("a", href=True)["href"])

#     position_1 = features[4].text
    
#     if position_1 in ("Centre-Back", "Left-Back", "Right-Back", "Defender"):
#         field_posit = "DEF"
#     elif position_1 in ("Defensive Midfield", "Central Midfield" , "Right Midfield" , "Left Midfield" , \
#     "Attacking Midfield" , "Midfield"):
#         field_posit = "MID"
#     elif position_1 in ("Left Winger" , "Right Winger" , "Centre-Forward" , "Second Striker" , "Forward"):
#         field_posit = "ATT"
#     else:
#         field_posit = "GLK"
#     try:
#         age_1 = int((features[5].text.split("(",)[-1])[:-1])
#     except:
#         age_1 = "-"

#     nat = features[6].img["alt"]

#     try:
#         Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
#     except:
#         Height_1 = "0"

#     foot_1 = features[8].text

#     dt_joined_1 = features[9].text
#     try:
#         dt_joined_1 = datetime.strptime(dt_joined_1, '%b %d, %Y').date()
#     except:
#         dt_joined_1 = "2019-01-01"

#     try:
#         prev_team_1 = features[10].img["alt"]
#     except:
#         prev_team_1 = "-"


#     contract_expires_1 = features[11].text
#     try:
#         contratc_expires_1 = datetime.strptime(contratc_expires_1, '%d.%m.%Y').date()
#     except:
#         contract_expires_1 = "2030-01-01"


#     try:
#         market_value_1 = get_value_us(features[12].text[:-2])
#     except:
#         market_value_1 = 0

#     name.append(strip_accents(name_1))
#     position.append(position_1)
#     f_posi.append(field_posit)
#     Age.append(age_1)
#     Nat.append(nat)
#     Height.append(Height_1)
#     foot.append(foot_1)
#     dt_joined.append(dt_joined_1)
#     prev_team.append(strip_accents(prev_team_1))
#     contract_expires.append(contract_expires_1)
#     market_value.append(market_value_1)
#     team_ID.append(team_id)
#     players_ID.append(player_id)
#     player_id = player_id+1

# #     time.sleep(randint(3,5))

# df = pd.DataFrame(list(zip(players_ID, team_ID, name, position,f_posi, Age,Nat, Height, foot,dt_joined,prev_team, contract_expires,\
#                            market_value,player_page)), 
#                   columns =["players_ID", "team_ID","name","position","field_position", "Age", "Nat","Height","foot",'dt_joined',"prev_team", \
#                             "contract_expires", "market_value","player_page" ])
# df['players_ID'] = df['players_ID'].astype(int)
# df['team_ID'] = df['team_ID'].astype(int)
# df['dt_joined'] = pd.to_datetime(df['dt_joined'])
# df['contract_expires'] = pd.to_datetime(df['contract_expires'])


# df


In [ ]:
# df.dtypes

In [ ]:
# from datetime import datetime

# df_copy = df.copy()

# for date in df_copy["contract_expires"]:
#     try:
#         date = datetime.strptime(date, '%d.%m.%Y').date()
#     except:
#         date = '-' 
#     print(date)


In [ ]:
# fla_copy = fla.copy()

# for date in fla_copy["dt_joined"]:
#     try:
#         date = datetime.strptime(date, '%b %d, %Y').date()
#     except:
#         date = "-"
#     print(date)

In [ ]:
# name_1 = features[2].find("img", alt=True)["alt"]
# position_1 = features[4].text
# DoB_1 = int((features[5].text.split("(",)[-1])[:-1])
# nat = features[6].img["alt"]
# Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
# foot_1 = features[8].text
# dt_joined_1 = features[9].text
# prev_team_1 = features[10].img["alt"]
# contract_expires_1 = features[11].text
# market_value_1 = get_value_us(features[12].text[:-2])

# print(name_1, position_1,DoB_1, nat, Height_1,foot_1,dt_joined_1,prev_team_1,contract_expires_1,market_value_1   )


In [ ]:
# pd.read_csv("data/teams/goias-esporte-clube_trmk.csv")

In [ ]:
# # Retrieve page with the requests module
# url = "https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1"
# html = requests.get(url, headers=headers)

In [ ]:
# # Create BeautifulSoup object; parse with 'html.parser'
# soup = BeautifulSoup(html.content, 'html.parser')
# htmltable = soup.find('table', class_= "items")
# text1 = htmltable.text
# text1

In [ ]:
# #Examine the results, then determine element that contains sought info
# print(soup.prettify())

In [ ]:
# results = htmltable.findAll("tr", class_ =["odd","even"])
# print(results[0])
# print(len(results))

In [ ]:
# # to Download imgs
# for img in logos:
#         with open(basename(img), "wb") as f:
#             f.write(requests.get(img).content)

In [ ]:
# names= []
# for result in results:
#     names.append(result.find("img", alt=True)["alt"])
    
# names

In [ ]:
## Extrating a table from HTML
# htmltable2 = str(htmltable)
# dfs = pd.read_html(htmltable2)
# df_clean = dfs[0][["Club.1","name.1", "ø age", "Total market value", "ø market value" ]]
# df_clean = df_clean.drop(df_clean.index[len(df_clean.index)-1])
# df = df_clean.set_axis(["Club","Squad", "Foreigners", "avg_market_value" , "total_market_value"], axis=1, inplace=False)

<td class="zentriert no-border-rechts">
    <a class="vereinprofil_tooltip" href="/los-angeles-football-club/startseite/verein/51828/saison_id/2019">
        <img alt="Los Angeles FC" class="tiny_wappen" src="https://tmssl.akamaized.net//images/wappen/tiny/51828.png?lm=1511112738" title=" "/>
    </a>
</td>
<td class="hauptlink no-border-links hide-for-small hide-for-pad">
    <a class="vereinprofil_tooltip" href="/los-angeles-football-club/startseite/verein/51828/saison_id/2019" id="51828">
        Los Angeles FC
    </a>
</td>
<td class="hauptlink no-border-links show-for-small show-for-pad">
    <a class="vereinprofil_tooltip" href="/los-angeles-football-club/startseite/verein/51828/saison_id/2019" id="51828">
        Los Angeles FC
    </a>
</td>
<td class="zentriert">
    <a href="/los-angeles-fc/kader/verein/51828/saison_id/2019" title="Los Angeles FC">
    24
    </a>
</td>
<td class="zentriert hide-for-small hide-for-pad">
    26,1
</td>
<td class="zentriert hide-for-pad hide-for-small">
    17
</td>
<td class="rechts hide-for-small hide-for-pad">
    <a href="/los-angeles-fc/kader/verein/51828/saison_id/2019" title="Los Angeles FC">
    €49.20m
    </a>
</td>
<td class="rechts hide-for-small hide-for-pad">
    €2.05m
</td>
<td class="rechts show-for-small show-for-pad nowrap">
    <a href="/los-angeles-fc/kader/verein/51828/saison_id/2019" title="Los Angeles FC">
        €49.20m
    </a>
</td>
<td class="rechts show-for-small show-for-pad nowrap">
    €2.05m
</td>


In [ ]:
# def get_value(x):
#     value = []
#     for char in x:
#         value.append(char)
#     float_value = "".join(value[1:-1])
#     return float(float_value)

In [ ]:
# get_value(fdf[6].text)

In [ ]:
# # URLs of page to be scraped
# bra = {"BRAZIL": {"leagues":[{"tier_1":'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1'},
#                  {"tier_2": "https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA2"}]}},
# us = {"UNITED_STATES": {"leagues":[{"tier_1": "https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1"},
#                   {"tier_2":"https://www.transfermarkt.com/usl-pro/startseite/wettbewerb/USL"}, 
#                   {"tier_3":"https://www.transfermarkt.com/usl-league-one/startseite/wettbewerb/USC3"}]}}
# urls= [bra, us]

In [ ]:
# def scrape_league_data(leagues_df):
#     team_id = 1
#     for index, row in leagues_df.iterrows():
#         url = row["league_link"]
#         league_name = row["league_name"]
#         tier = row["tier"]
#         country = row["country"]
#         league_id = row["league_ID"]
#         print(f"scraping: {country}_{tier}_{league_name}")
#         html = requests.get(url, headers=headers)
#         soup = BeautifulSoup(html.content, 'html.parser')
#         htmltable = soup.find('table', class_= "items")


#         results = htmltable.findAll("tr", class_ =["odd","even"])


#         team_ID = []
#         links = []
#         names = []
#         logos = []
#         squads = []
#         foreigners = []
#         total_MVs = []
#         avg_MVs = []
#         league_ID = []



#         for result in results:
#             features = result.findAll("td")
#             links.append(("https://www.transfermarkt.com"+result.find("a", href=True)\
#                           ["href"]+"/plus/1").replace("startseite", "kader"))
#             logo = result.find("img", src=True)["src"]
#             logo = logo.split("?")[0]
#             logo = logo.replace("tiny", "header")
#             logos.append(logo)
#             name = features[1].text
#             names.append(name)
#             squad = features[3].text
#             squads.append(squad)
#             foreigner = features[5].text
#             foreigners.append(foreigner)
#             total_MV = get_value_us(features[6].text)
#             total_MVs.append(total_MV)
#             avg_MV = get_value_us(features[7].text)
#             avg_MVs.append(avg_MV)
#             team_ID.append(team_id)
#             league_ID.append(league_id)
#             team_id = team_id + 1



#         # Create a Dataframe and export to a .csv file
#         df = pd.DataFrame(list(zip(team_ID, league_ID, names, squads, foreigners,avg_MVs, total_MVs, logos,links)), \
#     columns =["team_ID", "league_ID","club","squad", "foreigners", "avg_market_value_m", "total_MV_m",'Logo_img', "link_page"]) 
#         df['league_ID'] = df['league_ID'].astype(int)
#         df['team_ID'] = df['team_ID'].astype(int)

#         df.to_csv(f'data/{country}_{tier}_{league_name}_trmk.csv',index=False)


#         time.sleep(1)

In [ ]:
# def scrape_team_data(files):
#     player_id = 1
#     for filename in files:
#         print(filename)
#         time.sleep(5)
#         df_league = pd.read_csv(filename)
#         for index, row in df_league.iterrows():
#             team_id = row["team_ID"]
#             team = row["link_page"]
#             team_name = team.split('/')[-8]
#             print(team_name)
#             html = requests.get(team, headers=headers)
#             soup = BeautifulSoup(html.content, 'html.parser')
#             htmltable = soup.find('table', class_= "items")

#             results = htmltable.findAll("tr", class_ =["odd","even"])
#             name = []
#             player_page = []
#             position = []
#             Age = []
#             Nat = []
#             Height = []
#             foot = []
#             dt_joined = []
#             prev_team = []
#             contract_expires = []
#             market_value = []
#             team_ID = []
#             players_ID = []

#             for result in results:
#                 features = result.findAll("td")

#                 name_1 = features[2].find("img", alt=True)["alt"]

#                 player_page.append("https://www.transfermarkt.com" + features[3].find("a", href=True)["href"])

#                 position_1 = features[4].text

#                 try:
#                     age_1 = int((features[5].text.split("(",)[-1])[:-1])
#                 except:
#                     age_1 = "-"

#                 nat = features[6].img["alt"]

#                 try:
#                     Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
#                 except:
#                     Height_1 = "m"

#                 foot_1 = features[8].text

#                 dt_joined_1 = features[9].text
#                 try:
#                     dt_joined_1 = datetime.strptime(dt_joined_1, '%b %d, %Y').date()
#                 except:
#                     dt_joined_1 = "-"

#                 try:
#                     prev_team_1 = features[10].img["alt"]
#                 except:
#                     prev_team_1 = "-"


#                 contract_expires_1 = features[11].text
#                 try:
#                     contratc_expires_1 = datetime.strptime(contratc_expires_1, '%d.%m.%Y').date()
#                 except:
#                     contract_expires_1 = features[11].text


#                 try:
#                     market_value_1 = get_value_us(features[12].text[:-2])
#                 except:
#                     market_value_1 = 0

#                 name.append(name_1)
#                 position.append(position_1)
#                 Age.append(age_1)
#                 Nat.append(nat)
#                 Height.append(Height_1)
#                 foot.append(foot_1)
#                 dt_joined.append(dt_joined_1)
#                 prev_team.append(prev_team_1)
#                 contract_expires.append(contract_expires_1)
#                 market_value.append(market_value_1)
#                 team_ID.append(team_id)
#                 players_ID.append(player_id)
#                 player_id = player_id+1

#             df = pd.DataFrame(list(zip(players_ID, team_ID, name, position, Age,Nat, Height, foot,dt_joined,prev_team, contract_expires,\
#                                        market_value,player_page)), 
#                               columns =["players_ID", "team_ID","name","position", "Age", "Nat","Height","foot",'dt_joined',"prev_team", \
#                                         "contract_expires", "market_value","player_page" ])
#             df['players_ID'] = df['players_ID'].astype(int)
#             df['team_ID'] = df['team_ID'].astype(int)
            

#             df.to_csv(f"data/teams/{team_name}_trmk.csv" ,index=False)
#             time.sleep(1)